In [1]:
!git clone https://github.com/capogluuu/weatheralert.git

fatal: destination path 'weatheralert' already exists and is not an empty directory.


In [2]:
%cd weatheralert

/content/weatheralert


In [3]:
import pandas as pd
import requests

In [4]:
# read setting data and rewrite
setting_default_text = """# Location Information
latitude = 39.7456
longitude = -97.0892

# Application Settings
threshold_value = 80
check_in_frequency = 30

"""

a_file = open("settings.txt", "r")
list_of_lines = a_file.readlines()
a_file.close()

set_dict = {} 
for i in range(len(list_of_lines)):
  for j in range(len(list_of_lines[i].split("= "))):
    temp_variable = list_of_lines[i].split("= ")[j].strip()
    if(temp_variable == "latitude" or 
       temp_variable == "longitude" or
       temp_variable == "threshold_value" or
       temp_variable == "check_in_frequency"):
      set_dict[temp_variable] = list_of_lines[i].split("= ")[j+1].replace("\n","")

defaultdict = set_dict 

setting_text_template= """# Location Information
latitude = {latitude}
longitude = {longitude}

# Application Settings
threshold_value = {threshold_value}
check_in_frequency = {check_in_frequency}""".format(latitude = 3,longitude = 3,
                                                    threshold_value = 3,check_in_frequency = 3)
with open('settings.txt', "w") as myfile:
    myfile.write(setting_text_template)

In [5]:
response = requests.get(f'https://api.weather.gov/points/{39.7456},{-97.0892}')
data = response.json()
data

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   '@vocab': 'https://api.weather.gov/ontology#',
   'bearing': {'@type': 's:QuantitativeValue'},
   'city': 's:addressLocality',
   'county': {'@type': '@id'},
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'forecastGridData': {'@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'publicZone': {'@type': '@id'},
   's': 'https://schema.org/',
   'state': 's:addressRegion',
   'unit': 'http://codes.wmo.int/common/unit/',
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'value': {'@id': 's:value'},
   'wx': 'https://api.weather.gov/ontology#'}],
 'geometry': {'coordinates': [-97.0892, 39.7456], 'type': 'Point'},
 'id': 'https://api.weather.gov/points/39.7456,-97.0892',
 'properties': {'@id': 'https://api.weather.gov/points/39.7456,-97.08

In [6]:
response = requests.get(data["properties"]["forecastHourly"])
data1 = response.json()

In [ ]:
data1

In [8]:
df = pd.DataFrame(data1["properties"]["periods"])
df.head(2)

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,,2022-01-28T02:00:00-06:00,2022-01-28T03:00:00-06:00,False,24,F,None,10 mph,N,https://api.weather.gov/icons/land/night/bkn?s...,Mostly Cloudy,
1,2,,2022-01-28T03:00:00-06:00,2022-01-28T04:00:00-06:00,False,23,F,None,5 mph,N,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,


In [9]:
def convert_to_date(date_time_str, event = "undefined"):
  date_dict = {}
  date_dict[f"{event}_date"] = date_time_str.split("T")[0]
  date_time_str = date_time_str.split("T")[1]
  date_dict[f"{event}_hour"] = date_time_str.split("-")[0]
  return date_dict

In [10]:
convert_to_date('2022-01-28T01:00:00-06:00')

{'undefined_date': '2022-01-28', 'undefined_hour': '01:00:00'}

In [11]:
list_of_time_start = [convert_to_date(temp_text, "startTime") for temp_text in df["startTime"]]
list_of_time_end   = [convert_to_date(temp_text ,"endTime") for temp_text in df["endTime"]]

In [12]:
df2 = pd.DataFrame(list_of_time_start)
df3 = pd.DataFrame(list_of_time_end)
frames = [df, df2, df3]
result = pd.concat(frames,axis=1)
del result["number"]
del result["endTime"]

result = result[['name','startTime','startTime_date', 'startTime_hour',
       'endTime_date', 'endTime_hour','isDaytime', 'temperature', 'temperatureUnit',
       'temperatureTrend', 'windSpeed', 'windDirection', 'icon',
       'shortForecast', 'detailedForecast']]
result = result.head(9)

In [13]:
import pandas as pd

def create_database_table(df,
                          latitude = 3,
                          longitude = 3,
                          threshold_value = 3,
                          check_in_frequency = 3 ):
  print("1. step")
  timestamp = []
  longitude = []
  latitude = []
  first_forecast = [] 
  second_forecast = []
  third_forecast  = []
  ls_alert_generated = []
  ls_alert_id = []

  print("2. step")
  for i in range(len(df)-check_in_frequency):
    timestamp.append(df["startTime"][i])
    longitude.append(longitude)
    latitude.append(latitude)
    first_forecast.append(df["temperature"][i])
    second_forecast.append(df["temperature"][i+1])
    third_forecast.append(df["temperature"][i+2])
  
  print("3. step")
  #alert system
  for i in range(len(first_forecast)):
    alert_generated = False
    alert_id = "no_problem"
    large_range = third_forecast[i] - first_forecast[i]
    small_range_1 = second_forecast[i] - first_forecast[i]
    small_range_2 = third_forecast[i] - second_forecast[i]
    
    if( abs(large_range) >= threshold_value or
       abs(small_range_1) >= threshold_value or
       abs(small_range_2) >= threshold_value):
      
      alert_generated = True
      if(large_range < 0 ): alert_id= "low-temp"
      else : alert_id= "high-temp"
  
    
    ls_alert_generated.append(alert_generated)
    ls_alert_id.append(alert_id)

  print("4. step")
  temp_data = {'timestamp': timestamp, 
        'longitude': longitude,
        'latitude':latitude,
        'first_forecast':first_forecast,
        'second_forecast':second_forecast,
        'third_forecast':third_forecast,
        'alert_generated':ls_alert_generated,
        'alert_id':ls_alert_id
        }
  
  db_df = pd.DataFrame(data = temp_data)

  return db_df

In [ ]:
create_database_table(result)